# CSV parser

In [3]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},         #지금 이 partial_variables에 몇가지 인자가 들어갈 수 있는거지? 문자열, 함수, 출력지시자 등등...
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})

['Kimchi', 'Bibimbap', 'Bulgogi', 'Tteokbokki', 'Japchae']

In [14]:
prompt.invoke({"subject": "popular Korean cusine"}) #낮은 temperature와 상세한 가이드를 통해서 출력을 두드려팬듯

StringPromptValue(text='List five popular Korean cusine.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')

In [11]:
prompt = PromptTemplate(
    template="List five {subject}.\n",
    input_variables=["subject"],
)
chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})

['Here are five popular Korean dishes:',
 '1. **Kimchi** - A traditional side dish made from fermented vegetables',
 'primarily napa cabbage and radishes',
 'seasoned with various spices and ingredients.',
 '2. **Bibimbap** - A mixed rice dish topped with a variety of vegetables',
 'meat (often beef)',
 'a fried egg',
 'and gochujang (Korean chili paste).',
 '3. **Bulgogi** - Marinated beef that is grilled or stir-fried',
 'known for its sweet and savory flavor',
 'often served with rice and vegetables.',
 '4. **Tteokbokki** - Spicy stir-fried rice cakes',
 'typically made with gochujang',
 'fish cakes',
 'and vegetables',
 'popular as a street food snack.',
 '5. **Samgyeopsal** - Grilled pork belly',
 'usually served with lettuce leaves',
 'garlic',
 'and dipping sauces',
 'allowing diners to wrap the meat with various accompaniments.',
 'These dishes showcase the rich flavors and diverse ingredients characteristic of Korean cuisine.']

???? 대체...무슨 일이 일어난거임????  
실행시간을 봤을때 출력을 파싱한게 아니라 출력 자체가 저 csv 느낌으로 된건데 상상이 안된다

# JSON parset

In [19]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


In [20]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [22]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | model | output_parser

chain.invoke({"query": "Let me know how to cook Bibimbap"})

{'name': 'Bibimbap',
 'recipe': 'To cook Bibimbap, start by preparing the ingredients: cook rice and set aside. Sauté a variety of vegetables such as spinach, carrots, zucchini, and mushrooms separately in sesame oil, seasoning each with salt. Fry an egg sunny-side up. In a bowl, place a serving of rice, arrange the sautéed vegetables on top, add the fried egg, and drizzle with gochujang (Korean chili paste) and sesame oil. Mix everything together before eating.'}

유용한 내용이기는 한데 한국어에서도 안정적으로 동작할까?